# Ingest ephys session path on bl_new_acquisition.acquisition_sessions table

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


## Connection to DB

In [2]:
import datajoint as dj
import pandas as pd
import utility.path_utility as pu

bdata          = dj.create_virtual_module('bdata', 'bl_bdata')
shadow_acquisition = dj.create_virtual_module('shadow_acquisition', 'bl_shadow_acquisition')
new_acquisition = dj.create_virtual_module('new_acquisition', 'bl_new_acquisition')
new_lab = dj.create_virtual_module('new_lav', 'bl_new_lab')
ratinfo        = dj.create_virtual_module('ratinfo', 'bl_ratinfo')

Connecting alvaros@datajoint01.pni.princeton.edu:3306


## Get all sessions info

In [3]:
list_sessions_fields = ['sessid', 'session_rat', 'session_userid', 'session_rigid', 'session_date']
sessions_df = pd.DataFrame(new_acquisition.Sessions.fetch(*list_sessions_fields, order_by='sessid desc', as_dict=True))
list_contacts_fields = ['user_id', 'experimenter']
contact_df = pd.DataFrame(new_lab.Contacts.fetch(*list_contacts_fields, as_dict=True))
sessions_df = sessions_df.merge(contact_df, left_on='session_userid', right_on='user_id')
sessions_df['session_date'] = sessions_df['session_date'].astype('str')

## Create and find all paths from raw ephys sessions

In [4]:
ephys_root = dj.config['custom']['ephys_root_data_dir']
sessions_df['subject_path'] = sessions_df.apply(lambda x: pu.combine_str_path(ephys_root, [x['experimenter'], x['session_rat']]), axis=1)
sessions_df['session_path'] = sessions_df.apply(lambda x: pu.check_date_directory(x['subject_path'], x['session_date']), axis=1)

In [26]:
sessions_df.loc[sessions_df['sessid']==710898, :]

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,session_path
255117,710898,A230,abondy,205,2019-07-06,abondy,Adrian,//apps02.pni.princeton.edu/archive-labdata/bro...,Multiple ephys directory found


## Sessions with multiple possible ephys directories (needs experimenter to take action)

In [5]:
multiple_ephys_dir_df = sessions_df.loc[sessions_df['session_path'] == pu.path_not_found_dict['mulitple_ephys'], :]
multiple_ephys_dir_df = multiple_ephys_dir_df.reset_index(drop=True)
multiple_ephys_dir_df['session_path'] = multiple_ephys_dir_df.apply(lambda x: pu.check_date_directory(x['subject_path'], x['session_date'], return_multiple=True), axis=1)
multiple_ephys_dir_df

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,session_path
0,797011,T274,zhihaol,202,2020-12-04,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
1,796369,T274,zhihaol,202,2020-11-30,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
2,793486,T240,zhihaol,202,2020-11-08,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
3,793314,T240,zhihaol,202,2020-11-07,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
4,793313,T240,zhihaol,202,2020-11-07,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
...,...,...,...,...,...,...,...,...,...
174,344848,M071,kevinjmiller,204,2015-03-24,kevinjmiller,Kevin,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
175,342019,M071,kevinjmiller,204,2015-03-04,kevinjmiller,Kevin,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
176,324137,W035,athena.akrami,201,2014-11-07,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...
177,323841,W035,athena.akrami,201,2014-11-05,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,[//apps02.pni.princeton.edu/archive-labdata/br...


### Example session with multiple folders from same session

In [43]:
multiple_ephys_dir_df.loc[0, 'sessid']

797011

## Multiple session with same ephys directory

In [7]:
#Filter sessions with an ephys folder found
ephys_sessions_df = sessions_df.loc[~sessions_df['session_path'].isin(pu.path_not_found_dict.values()), :]
ephys_sessions_df = ephys_sessions_df.reset_index(drop=True)

#Find sessions with same ephys folder
idx_repeated = ephys_sessions_df.duplicated(keep=False, subset=['session_path'])
mult_sess_same_ephys_df = ephys_sessions_df[idx_repeated].copy()
mult_sess_same_ephys_df = mult_sess_same_ephys_df.reset_index(drop=True)
mult_sess_same_ephys_df

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,session_path
0,797171,T305,zhihaol,202,2020-12-05,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
1,797169,T305,zhihaol,202,2020-12-05,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
2,796531,T274,zhihaol,202,2020-12-01,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
3,796530,T274,zhihaol,202,2020-12-01,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
4,792705,T240,zhihaol,202,2020-11-02,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
...,...,...,...,...,...,...,...,...,...
153,323618,M070,kevinjmiller,204,2014-11-04,kevinjmiller,Kevin,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
154,322957,M070,kevinjmiller,13,2014-10-31,kevinjmiller,Kevin,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
155,322933,M070,kevinjmiller,204,2014-10-31,kevinjmiller,Kevin,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
156,326545,W055,athena.akrami,201,2014-11-21,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...


In [42]:
mult_sess_same_ephys_df.loc[0, 'session_path']

'//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T305/T305_2020_12_05b_g0'

## Sessions with unequivocally found ephys folder

In [8]:
corr_ephys_sessions_df = ephys_sessions_df[~idx_repeated].copy()
corr_ephys_sessions_df = corr_ephys_sessions_df.reset_index(drop=True)
corr_ephys_sessions_df

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,session_path
0,798899,T274,zhihaol,202,2020-12-17,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
1,798897,T274,zhihaol,202,2020-12-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
2,798742,T274,zhihaol,202,2020-12-15,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
3,798586,T274,zhihaol,202,2020-12-14,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
4,798430,T274,zhihaol,202,2020-12-13,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
...,...,...,...,...,...,...,...,...,...
908,323955,W055,athena.akrami,201,2014-11-06,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
909,323873,W055,athena.akrami,201,2014-11-05,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
910,323448,W035,athena.akrami,201,2014-11-03,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...
911,323153,W035,athena.akrami,201,2014-11-01,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...


## Rename path columns and add other columns for ingestion

In [34]:
acquisition_session_df = corr_ephys_sessions_df.copy()
acquisition_session_df['acquisition_type'] = 'ephys'
acquisition_session_df['acquisition_raw_abs_path'] = acquisition_session_df['session_path']
acquisition_session_df['acquisition_raw_rel_path'] = acquisition_session_df['session_path'].str.replace(dj.config['custom']['ephys_root_data_dir'], '', regex=False)
acquisition_session_df = acquisition_session_df.drop(columns=['session_path'], axis=1)
acquisition_session_df

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,acquisition_type,acquisition_raw_abs_path,acquisition_raw_rel_path
0,798899,T274,zhihaol,202,2020-12-17,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_17_g0
1,798897,T274,zhihaol,202,2020-12-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_16_g0
2,798742,T274,zhihaol,202,2020-12-15,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_15_g0
3,798586,T274,zhihaol,202,2020-12-14,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_14_g0
4,798430,T274,zhihaol,202,2020-12-13,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_13c_g0
...,...,...,...,...,...,...,...,...,...,...,...
908,323955,W055,athena.akrami,201,2014-11-06,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Athena/W055/2014-11-06_13-18-08
909,323873,W055,athena.akrami,201,2014-11-05,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Athena/W055/2014-11-05_17-00-09
910,323448,W035,athena.akrami,201,2014-11-03,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Athena/W035/2014-11-03_14-50-05
911,323153,W035,athena.akrami,201,2014-11-01,athena.akrami,Athena,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Athena/W035/2014-11-01_14-24-08


## Find sorted files for found raw sessions

In [37]:
cluster_root = dj.config['custom']['clustering_root_data_dir']
acquisition_session_df['subject_clutser_path'] = acquisition_session_df.apply(lambda x: pu.combine_str_path(cluster_root, [x['experimenter'], x['session_rat']]), axis=1)
acquisition_session_df['acquisition_post_abs_path'] = acquisition_session_df.apply(lambda x: pu.check_date_directory(x['subject_clutser_path'], x['session_date']), axis=1)
acquisition_session_df['acquisition_post_rel_path'] = acquisition_session_df['acquisition_post_abs_path'].str.replace(dj.config['custom']['clustering_root_data_dir'], '', regex=False)

#Filter sessions with a cluster folder folder found
acquisition_session_df = acquisition_session_df.loc[~acquisition_session_df['acquisition_post_abs_path'].isin(pu.path_not_found_dict.values()), :]
acquisition_session_df = acquisition_session_df.reset_index(drop=True)
acquisition_session_df

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,acquisition_type,acquisition_raw_abs_path,acquisition_raw_rel_path,subject_clutser_path,acquisition_post_abs_path,acquisition_post_rel_path
0,798899,T274,zhihaol,202,2020-12-17,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_17_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_17
1,798897,T274,zhihaol,202,2020-12-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_16_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_16
2,798742,T274,zhihaol,202,2020-12-15,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_15_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_15
3,798586,T274,zhihaol,202,2020-12-14,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_14_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_14
4,798430,T274,zhihaol,202,2020-12-13,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_13c_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_13c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,701708,A242,abondy,205,2019-06-05,abondy,Adrian,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Adrian/A242/2019-06-05_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Adrian/A242/A242_2019_06_05
305,701539,A242,abondy,205,2019-06-04,abondy,Adrian,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Adrian/A242/2019-06-04_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Adrian/A242/A242_2019_06_04
306,701531,A242,abondy,205,2019-06-03,abondy,Adrian,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Adrian/A242/2019-06-03_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Adrian/A242/A242_2019_06_03
307,700851,A242,abondy,205,2019-05-31,abondy,Adrian,//apps02.pni.princeton.edu/archive-labdata/bro...,ephys,//apps02.pni.princeton.edu/archive-labdata/bro...,Adrian/A242/2019-05-31_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Adrian/A242/A242_2019_05_31


## Ingest found sessions

In [38]:
acquisition_session_df_final = acquisition_session_df[new_acquisition.AcquisitionSessions.heading.names]
dict_keys = acquisition_session_df_final.to_dict(orient='records')
for i in dict_keys:
    new_acquisition.AcquisitionSessions.insert1(i, replace=True)

In [39]:
new_acquisition.AcquisitionSessions()

sessid Unique number assigned to each training session,session_rat ratname inherited from rats table,session_userid rat owner inherited from contacts table,session_rigid rig id number inherited from riginfo table,acquisition_type ephys or imaging,acquisition_raw_abs_path absoulte path of raw files,acquisition_raw_rel_path relative path (from ephys or imaging root dir),acquisition_post_abs_path absoulte path of post processing file (clustered/segmented),acquisition_post_rel_path relative path (from ephys or imaging clustering/segmentation root dir)
585444,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_03_16,Thomas/T170/T170_2018_03_16,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_03_16_600s,Thomas/T170/T170_2018_03_16_600s
585488,T173,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_bank1_2018_03_16,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T173/T173_2018_03_16_bank1_600s,Thomas/T173/T173_2018_03_16_bank1_600s
586335,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19
586376,T173,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_bank0_2018_03_19,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T173/T173_2018_03_19_bank0_600s,Thomas/T173/T173_2018_03_19_bank0_600s
590577,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05
592370,T173,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T173/T173_bank1_2018_04_11,Thomas/T173/T173_bank1_2018_04_11,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T173/T173_2018_04_11_bank1_600s,Thomas/T173/T173_2018_04_11_bank1_600s
592637,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_04_12,Thomas/T170/T170_2018_04_12,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_04_12,Thomas/T170/T170_2018_04_12
592934,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_04_13,Thomas/T170/T170_2018_04_13,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_04_13,Thomas/T170/T170_2018_04_13
593790,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_04_16,Thomas/T170/T170_2018_04_16,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_04_16,Thomas/T170/T170_2018_04_16
594103,T170,zhihaol,202,ephys,//apps02.pni.princeton.edu/archive-labdata/brody/RATTER/PhysData/Raw/Thomas/T170/T170_2018_04_17,Thomas/T170/T170_2018_04_17,//bucket.pni.princeton.edu/brody/RATTER/PhysData/NP_sorted/Thomas/T170/T170_2018_04_17_600s,Thomas/T170/T170_2018_04_17_600s
